In [17]:
from models import SEAsiaCFR, BrazilCFR, MineNetBack, ActualNetback
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("dark_background")

In [2]:
simulations = 20000

# BRAZIL

In [7]:
self = BrazilCFR()

In [8]:
month = 8
year = 2022

In [13]:
self.predict(month, year)

Predicting Brazil >> 8 / 2022...


100%|███████████████████████████████████████████████████| 20000/20000 [00:03<00:00, 5477.54it/s]


**************************************************
        Predictions
count  20000.000000
mean     897.709395
std       62.459763
min      700.278014
25%      852.922910
50%      897.679735
75%      942.386660
max     1085.127525
**************************************************
Max Density -->>> $898.48


898.48

# SEASIA

In [14]:
self = SEAsiaCFR()

In [16]:
self.predict(month,year)

Predicting SE Asia >> 8 / 2022...


100%|██████████████████████████████████████████████████| 20000/20000 [00:01<00:00, 10323.26it/s]


**************************************************
        Predictions
count  20000.000000
mean     947.036814
std       67.319772
min      772.056112
25%      898.068982
50%      947.303002
75%      994.684851
max     1115.589014
**************************************************
Max Density -->>> $952.42


952.42

# MINE NETBACK

In [19]:
self = MineNetBack()

In [20]:
self.predict(month, year)

Predicting MineNetback >> 8 / 2022...
Predicting SE Asia >> 8 / 2022...


100%|██████████████████████████████████████████████████| 20000/20000 [00:01<00:00, 10124.12it/s]


**************************************************
        Predictions
count  20000.000000
mean     946.415243
std       67.184073
min      775.222766
25%      898.261013
50%      946.077124
75%      994.455621
max     1115.702276
**************************************************
Max Density -->>> $946.48
Predicting Brazil >> 8 / 2022...


100%|███████████████████████████████████████████████████| 20000/20000 [00:03<00:00, 5771.82it/s]


**************************************************
        Predictions
count  20000.000000
mean     898.450886
std       62.649756
min      694.246891
25%      853.565642
50%      898.441779
75%      943.505402
max     1087.146558
**************************************************
Max Density -->>> $897.38


100%|██████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 20043.30it/s]


**************************************************
        Predictions
count  20000.000000
mean     750.182831
std       41.664550
min      612.128390
25%      720.930090
50%      750.011298
75%      779.422525
max      877.399295
**************************************************
Max Density -->>> $747.15


747.15

In [ ]:
# get DATA
naturalgas = self.get_natural_gas
gdp = self.get_gdp
eurusd = self.get_eurusd
fertprod = self.get_total_fertilizer_production
seasia_cfr = self.get_SEAsiaCFR

dm = self.monthly_dummy()
const = self.const

In [ ]:
# len(fao.dropna())
# len(eurusd.dropna())
# len(fertprod.dropna())
# len(inflation.dropna())
# len(gdp.dropna())
# len(brazil_cfr.dropna())

# LATEST ENTRY ONLY

In [ ]:
x = seasia_cfr
x[(x.index.month==self.month) & (x.index.year==self.year)].iloc[-1]

In [ ]:
# Natural Gas
n = naturalgas[(naturalgas.index.month==self.month) & (naturalgas.index.year==self.year)].iloc[-1]
# GDP (based on current month)
g = gdp[(gdp.index.month==self.month) & (gdp.index.year==self.year)].iloc[-1]
# EURUSD
e = eurusd[(eurusd.index.month==self.month) & (eurusd.index.year==self.year)].iloc[-1]
# FERTILIZER PRODUCTION
fert = fertprod[(fertprod.index.month==self.month) & (fertprod.index.year==self.year)].iloc[-1]
# BRAZIL CFR
s = seasia_cfr.iloc[-1]

In [ ]:
seasia_latest_cfr = []
for i in tqdm(range(simulations)):
    ng0m = np.random.triangular(n.NGHHUUS-n.std_6,n.NGHHUUS,n.NGHHUUS+n.std_6)
    g0m = np.random.triangular(g.GDPQXUS-g.std_48,g.GDPQXUS,g.GDPQXUS+g.std_48)
    e0m = np.random.triangular(e['Adj Close']-e.std_3,e['Adj Close'],e['Adj Close']+e.std_3)
    fert0m = np.random.triangular(fert['Total Fertilizer Production']-fert.std_60,fert['Total Fertilizer Production'],fert['Total Fertilizer Production']+fert.std_60)
    s0m = np.random.triangular(s.SEAsia-s.std_12,s.SEAsia,s.SEAsia+s.std_12)
    result =dm + const + self.HHNaturalGasPrice* ng0m + self.USGDP*g0m + self.USDEURO*e0m + self.TotalFertilizerProduction*fert0m + self.SEAsia_1 *s0m
    seasia_latest_cfr.append(result)

In [ ]:
pred_df = pd.DataFrame(seasia_latest_cfr,columns=['Predictions'])

In [ ]:
pred_df

In [ ]:
pred_df.describe()

In [ ]:
pred_df.plot(kind='hist',bins=100,title=f'SEAsia CFR - {simulations} Iterations');

In [ ]:
self.kde_max_density(pred_df)